In [1]:
import numpy as np
from math import exp
from decimal import Decimal

In [77]:
class System(object):
    
        def __init__(self, num_particles, pfun, particles=None, dim=3, init_range=50, delta=10**-5):
            self.pfun = pfun
            self.num_particles = num_particles
            self.dim = dim
            self.delta = 10**-5
            if particles:
                self.particles = particles
            else:
                self.init_range = init_range
                self.particles = [np.random.rand(self.dim) * self.init_range
                                  for num in range(num_particles)]
            self.get_potential()
            self.get_numeric_grad()          

        def get_potential(self):
            self.potential = 0.
            for ind, i in enumerate(self.particles):
                for j in self.particles[ind+1:]:
                    self.potential += self.pfun(i, j)
        
        def get_active_potential(self, particle, ignore=None):
            active_potential = 0.
            for i, neighbour in enumerate(self.particles):
                if i == ignore:
                    continue
                active_potential += self.pfun(particle, neighbour)
            return active_potential
        
        def get_numeric_grad(self):
            self.gradients = list()
            delta_matrix = np.identity(self.dim) * self.delta
            for i, particle in enumerate(self.particles):
                gradient = np.zeros(3)
                for ind in range(self.dim):
                    pot_plus = self.get_active_potential(particle + delta_matrix[ind], ignore=i)
                    pot_minus = self.get_active_potential(particle - delta_matrix[ind], ignore=i)
                    gradient[ind] = 1/(2*self.delta) * (pot_plus - pot_minus)
                self.gradients.append(gradient) 
        
        def to_XYZ(self, path):
            with open(path, 'w') as f:
                f.writeline(`self.num_particles`)
                f.writeline("Geometry of system, ")
                for i, particle in enumerate(self.particles):
                    f.writeline("Particle%i %.4f %.4f %.4f" % (i, particle[0], particle[1], particle[2]))
                
        def __str__(self):
            if self.dim != 3:
                print "Can't print for dimensions other than 3"
                return
            output = 'particle\tx\ty\tz\n'
            for i, particle in enumerate(self.particles):
                output += "%i\t\t%.3f\t%.3f\t%.3f\n" % (i+1, particle[0], particle[1], particle[2])
            return output
        
        @staticmethod
        def Lennard_Jones(sigma, epsilon):
            def Lennard_Jones_pfun(nparr1, nparr2):
                r = np.linalg.norm(nparr1 - nparr2)
                return 4*epsilon*((sigma / r)**12 - (sigma / r)**6)
            return Lennard_Jones_pfun

        @staticmethod
        def Morse(sigma, De, re):
            def Morse_pfun(nparr1, nparr2):
                r = np.linalg.norm(nparr1 = nparr2)
                return De*(1 - exp(-(r-re)/sigma))**2
            return Morse_pfun                         

In [208]:
class Optimiser(object):
        
    def __init__(self, system, lambd=1, epsilon=10**-7):
        self.system = system
        self.lambd = lambd
        self.epsilon = epsilon
        self.steps = 0
    
    def _update(self):
        self.steps += 1
        for particle, grad in zip(self.system.particles, self.system.gradients):
            particle -= self.lambd * grad
        self.system.get_numeric_grad()
        self.system.get_potential()
        
    def jump_forward(self, jump_size):
        for step in range(jump_size):
            self._update()
    
    
    def optimise(self, jump_size=10, verbose='extra'):
        jumps = 0
        diff = float("inf")
        
        while (abs(diff) > self.epsilon):
            prev_potential = self.system.potential
            self.jump_forward(jump_size)
            jumps += 1
            diff = self.system.potential - prev_potential
            if verbose == 'extra':
                if jumps % 1 == 0:
                    print 'E (%s jumps) = %.3E, change = %.3E' \
                    % (jumps, Decimal(self.system.potential), Decimal(diff))
        
        if verbose:
            print 'Converged after %i jumps (%i iterations)\n' % (jumps, jumps*jump_size)
            print(self.system)  

In [210]:
sys = System(7, System.Lennard_Jones(1, 10))
optimiser = Optimiser(sys)
optimiser.optimise()

E (1 jumps) = -1.754E-04, change = -1.199E-07
E (2 jumps) = -1.755E-04, change = -1.201E-07
E (3 jumps) = -1.757E-04, change = -1.203E-07
E (4 jumps) = -1.758E-04, change = -1.205E-07
E (5 jumps) = -1.759E-04, change = -1.207E-07
E (6 jumps) = -1.760E-04, change = -1.209E-07
E (7 jumps) = -1.761E-04, change = -1.211E-07
E (8 jumps) = -1.763E-04, change = -1.213E-07
E (9 jumps) = -1.764E-04, change = -1.215E-07
E (10 jumps) = -1.765E-04, change = -1.217E-07
E (11 jumps) = -1.766E-04, change = -1.219E-07
E (12 jumps) = -1.768E-04, change = -1.221E-07
E (13 jumps) = -1.769E-04, change = -1.223E-07
E (14 jumps) = -1.770E-04, change = -1.225E-07
E (15 jumps) = -1.771E-04, change = -1.227E-07
E (16 jumps) = -1.772E-04, change = -1.229E-07
E (17 jumps) = -1.774E-04, change = -1.231E-07
E (18 jumps) = -1.775E-04, change = -1.233E-07
E (19 jumps) = -1.776E-04, change = -1.236E-07
E (20 jumps) = -1.777E-04, change = -1.238E-07
E (21 jumps) = -1.779E-04, change = -1.240E-07
E (22 jumps) = -1.780E

E (175 jumps) = -1.998E-04, change = -1.643E-07
E (176 jumps) = -2.000E-04, change = -1.646E-07
E (177 jumps) = -2.002E-04, change = -1.649E-07
E (178 jumps) = -2.003E-04, change = -1.653E-07
E (179 jumps) = -2.005E-04, change = -1.656E-07
E (180 jumps) = -2.007E-04, change = -1.659E-07
E (181 jumps) = -2.008E-04, change = -1.663E-07
E (182 jumps) = -2.010E-04, change = -1.666E-07
E (183 jumps) = -2.012E-04, change = -1.669E-07
E (184 jumps) = -2.013E-04, change = -1.673E-07
E (185 jumps) = -2.015E-04, change = -1.676E-07
E (186 jumps) = -2.017E-04, change = -1.679E-07
E (187 jumps) = -2.018E-04, change = -1.683E-07
E (188 jumps) = -2.020E-04, change = -1.686E-07
E (189 jumps) = -2.022E-04, change = -1.690E-07
E (190 jumps) = -2.024E-04, change = -1.693E-07
E (191 jumps) = -2.025E-04, change = -1.696E-07
E (192 jumps) = -2.027E-04, change = -1.700E-07
E (193 jumps) = -2.029E-04, change = -1.703E-07
E (194 jumps) = -2.030E-04, change = -1.707E-07
E (195 jumps) = -2.032E-04, change = -1.

E (349 jumps) = -2.346E-04, change = -2.424E-07
E (350 jumps) = -2.348E-04, change = -2.431E-07
E (351 jumps) = -2.350E-04, change = -2.437E-07
E (352 jumps) = -2.353E-04, change = -2.443E-07
E (353 jumps) = -2.355E-04, change = -2.449E-07
E (354 jumps) = -2.358E-04, change = -2.455E-07
E (355 jumps) = -2.360E-04, change = -2.462E-07
E (356 jumps) = -2.363E-04, change = -2.468E-07
E (357 jumps) = -2.365E-04, change = -2.474E-07
E (358 jumps) = -2.368E-04, change = -2.481E-07
E (359 jumps) = -2.370E-04, change = -2.487E-07
E (360 jumps) = -2.373E-04, change = -2.493E-07
E (361 jumps) = -2.375E-04, change = -2.500E-07
E (362 jumps) = -2.378E-04, change = -2.506E-07
E (363 jumps) = -2.380E-04, change = -2.513E-07
E (364 jumps) = -2.383E-04, change = -2.519E-07
E (365 jumps) = -2.385E-04, change = -2.526E-07
E (366 jumps) = -2.388E-04, change = -2.532E-07
E (367 jumps) = -2.390E-04, change = -2.539E-07
E (368 jumps) = -2.393E-04, change = -2.546E-07
E (369 jumps) = -2.395E-04, change = -2.

E (522 jumps) = -2.887E-04, change = -4.058E-07
E (523 jumps) = -2.891E-04, change = -4.072E-07
E (524 jumps) = -2.895E-04, change = -4.087E-07
E (525 jumps) = -2.899E-04, change = -4.101E-07
E (526 jumps) = -2.904E-04, change = -4.116E-07
E (527 jumps) = -2.908E-04, change = -4.131E-07
E (528 jumps) = -2.912E-04, change = -4.146E-07
E (529 jumps) = -2.916E-04, change = -4.161E-07
E (530 jumps) = -2.920E-04, change = -4.176E-07
E (531 jumps) = -2.924E-04, change = -4.191E-07
E (532 jumps) = -2.929E-04, change = -4.206E-07
E (533 jumps) = -2.933E-04, change = -4.221E-07
E (534 jumps) = -2.937E-04, change = -4.237E-07
E (535 jumps) = -2.941E-04, change = -4.252E-07
E (536 jumps) = -2.946E-04, change = -4.268E-07
E (537 jumps) = -2.950E-04, change = -4.284E-07
E (538 jumps) = -2.954E-04, change = -4.300E-07
E (539 jumps) = -2.958E-04, change = -4.315E-07
E (540 jumps) = -2.963E-04, change = -4.331E-07
E (541 jumps) = -2.967E-04, change = -4.348E-07
E (542 jumps) = -2.971E-04, change = -4.

E (704 jumps) = -4.011E-04, change = -9.596E-07
E (705 jumps) = -4.021E-04, change = -9.659E-07
E (706 jumps) = -4.030E-04, change = -9.723E-07
E (707 jumps) = -4.040E-04, change = -9.788E-07
E (708 jumps) = -4.050E-04, change = -9.853E-07
E (709 jumps) = -4.060E-04, change = -9.919E-07
E (710 jumps) = -4.070E-04, change = -9.986E-07
E (711 jumps) = -4.080E-04, change = -1.005E-06
E (712 jumps) = -4.090E-04, change = -1.012E-06
E (713 jumps) = -4.100E-04, change = -1.019E-06
E (714 jumps) = -4.110E-04, change = -1.026E-06
E (715 jumps) = -4.121E-04, change = -1.033E-06
E (716 jumps) = -4.131E-04, change = -1.040E-06
E (717 jumps) = -4.142E-04, change = -1.048E-06
E (718 jumps) = -4.152E-04, change = -1.055E-06
E (719 jumps) = -4.163E-04, change = -1.063E-06
E (720 jumps) = -4.173E-04, change = -1.070E-06
E (721 jumps) = -4.184E-04, change = -1.078E-06
E (722 jumps) = -4.195E-04, change = -1.085E-06
E (723 jumps) = -4.206E-04, change = -1.093E-06
E (724 jumps) = -4.217E-04, change = -1.

E (877 jumps) = -8.823E-04, change = -9.163E-06
E (878 jumps) = -8.917E-04, change = -9.444E-06
E (879 jumps) = -9.015E-04, change = -9.738E-06
E (880 jumps) = -9.115E-04, change = -1.005E-05
E (881 jumps) = -9.219E-04, change = -1.037E-05
E (882 jumps) = -9.326E-04, change = -1.072E-05
E (883 jumps) = -9.437E-04, change = -1.108E-05
E (884 jumps) = -9.551E-04, change = -1.146E-05
E (885 jumps) = -9.670E-04, change = -1.186E-05
E (886 jumps) = -9.793E-04, change = -1.229E-05
E (887 jumps) = -9.920E-04, change = -1.274E-05
E (888 jumps) = -1.005E-03, change = -1.322E-05
E (889 jumps) = -1.019E-03, change = -1.373E-05
E (890 jumps) = -1.033E-03, change = -1.427E-05
E (891 jumps) = -1.048E-03, change = -1.484E-05
E (892 jumps) = -1.064E-03, change = -1.545E-05
E (893 jumps) = -1.080E-03, change = -1.610E-05
E (894 jumps) = -1.096E-03, change = -1.680E-05
E (895 jumps) = -1.114E-03, change = -1.755E-05
E (896 jumps) = -1.132E-03, change = -1.835E-05
E (897 jumps) = -1.152E-03, change = -1.

In [225]:
sys = System(7, System.Lennard_Jones(1, 1))
print sys.gradients

[[nan, nan, nan], [nan, nan, nan], [nan, nan, nan], [nan, nan, nan], [nan, nan, nan], [nan, nan, nan], [nan, nan, nan]]


In [167]:
test.get_active_potential([12.1, 41.4, 48.5])

In [147]:
test.get_active_potential(np.random.rand(3))
print test.gradients

[[nan, nan, nan], [nan, nan, nan], [nan, nan, nan], [nan, nan, nan], [nan, nan, nan], [nan, nan, nan], [nan, nan, nan]]


In [133]:
test_fun = System.Lennard_Jones(1, 1)
test_fun(test.particles[0], test.particles[1]) + 0.1

0.099999999515338764